In [1]:
import numpy
import json
import csv
import pandas
import requests
import calendar
from datetime import datetime

In [2]:
cars_data = pandas.read_csv('cars_input.csv', encoding='big5')
cars_data

,資料年月,縣市,燃料別,車種類別,數量
0,201209,臺北市,汽油,大客車,39
1,201209,臺北市,汽油,大貨車,26
2,201209,臺北市,汽油,小客車,635193
3,201209,臺北市,汽油,小貨車,36222
4,201209,臺北市,汽油,特種車,4797
...,...,...,...,...,...
11097,202210,臺北市,電能,機車輕型,23619
11098,202210,臺北市,電能(增程),機車輕型,0
11099,202210,臺北市,電能/汽油,機車輕型,0
11100,202210,臺北市,電能/柴油,機車輕型,0


In [3]:
latest_cars_data = cars_data[cars_data['資料年月'] == 202210]
latest_cars_data

,資料年月,縣市,燃料別,車種類別,數量
11011,202210,臺北市,汽油,大客車,15
11012,202210,臺北市,汽油(油電),大客車,0
11013,202210,臺北市,汽油/LPG,大客車,0
11014,202210,臺北市,汽油/電能,大客車,0
11015,202210,臺北市,柴油,大客車,5999
...,...,...,...,...,...
11097,202210,臺北市,電能,機車輕型,23619
11098,202210,臺北市,電能(增程),機車輕型,0
11099,202210,臺北市,電能/汽油,機車輕型,0
11100,202210,臺北市,電能/柴油,機車輕型,0


In [4]:
latest_cars_data.insert(3, '燃料類型', '')

In [5]:
energy_list = {
	'汽油': '化石燃料',
	'汽油(油電)': '混和動力',
	'汽油/LPG': '化石燃料',
	'汽油/電能': '混和動力',
	'柴油': '化石燃料',
	'柴油(油電)': '混和動力',
	'柴油/電能': '混和動力',
	'液化石油氣': '化石燃料',
	'電能': '非化石燃料',
	'電能(增程)': '非化石燃料',
	'電能/汽油': '混和動力',
	'電能/柴油': '混和動力',
	'總計': '總計',
}

for i in range(len(latest_cars_data)):
	latest_cars_data.iat[i, 3] = energy_list[ latest_cars_data.iat[i, 2] ]
latest_cars_data

,資料年月,縣市,燃料別,燃料類型,車種類別,數量
11011,202210,臺北市,汽油,化石燃料,大客車,15
11012,202210,臺北市,汽油(油電),混和動力,大客車,0
11013,202210,臺北市,汽油/LPG,化石燃料,大客車,0
11014,202210,臺北市,汽油/電能,混和動力,大客車,0
11015,202210,臺北市,柴油,化石燃料,大客車,5999
...,...,...,...,...,...,...
11097,202210,臺北市,電能,非化石燃料,機車輕型,23619
11098,202210,臺北市,電能(增程),非化石燃料,機車輕型,0
11099,202210,臺北市,電能/汽油,混和動力,機車輕型,0
11100,202210,臺北市,電能/柴油,混和動力,機車輕型,0


In [6]:
latest_cars_data = latest_cars_data[['燃料類型', '車種類別', '數量']]
latest_cars_data = latest_cars_data[ latest_cars_data['數量'] > 0 ]
latest_cars_data = latest_cars_data[ latest_cars_data['燃料類型'] != '總計' ]
latest_cars_data

,燃料類型,車種類別,數量
11011,化石燃料,大客車,15
11015,化石燃料,大客車,5999
11017,混和動力,大客車,20
11019,非化石燃料,大客車,126
11024,化石燃料,大貨車,6
11028,化石燃料,大貨車,7878
11029,混和動力,大貨車,9
11030,混和動力,大貨車,14
11037,化石燃料,小客車,632521
11038,混和動力,小客車,41317


In [7]:
latest_oil_cars_data = latest_cars_data[ latest_cars_data['燃料類型'] == '化石燃料' ]
latest_hybrid_cars_data = latest_cars_data[ latest_cars_data['燃料類型'] == '混和動力' ]
latest_non_oil_cars_data = latest_cars_data[ latest_cars_data['燃料類型'] == '非化石燃料' ]

In [8]:
latest_cars_data_total = latest_cars_data['數量'].sum()
latest_oil_cars_total = latest_oil_cars_data['數量'].sum()
latest_hybrid_cars_total = latest_hybrid_cars_data['數量'].sum()
latest_non_oil_cars_total = latest_non_oil_cars_data['數量'].sum()

In [9]:
result = {
	'data': [{
		'name': '車輛燃料圖', 
		'data': []
	}]
}

result['data'][0]['data'].append( {'x': '化石燃料', 'y': latest_oil_cars_total} )
result['data'][0]['data'].append( {'x': '混和動力', 'y': latest_hybrid_cars_total} )
result['data'][0]['data'].append( {'x': '非化石燃料', 'y': latest_non_oil_cars_total} )

result

{'data': [{'name': '車輛燃料圖',
   'data': [{'x': '化石燃料', 'y': 1632716},
    {'x': '混和動力', 'y': 59703},
    {'x': '非化石燃料', 'y': 81290}]}]}

In [10]:
with open('cars_chart.json', 'w', encoding='utf-8') as file:
	file.write( json.dumps(result, ensure_ascii=False, indent=4, default=int) )

# 歷史資料

In [11]:
past_cars_data = cars_data
past_cars_data.insert(3, '燃料類型', '')
for i in range(len(past_cars_data)):
	past_cars_data.iat[i, 3] = energy_list[ past_cars_data.iat[i, 2] ]

In [12]:
past_cars_data = past_cars_data[['資料年月', '燃料類型', '數量']]
# past_cars_data = past_cars_data[ latest_cars_data['數量'] > 0 ]
past_cars_data = past_cars_data[ past_cars_data['燃料類型'] != '總計' ]

In [13]:
past_oil_cars_data = past_cars_data[ past_cars_data['燃料類型'] == '化石燃料' ]
# past_hybrid_cars_data = past_cars_data[ past_cars_data['燃料類型'] == '混和動力' ]
# past_non_oil_cars_data = past_cars_data[ past_cars_data['燃料類型'] == '非化石燃料' ]

In [14]:
past_oil_cars_data['資料年月'] = pandas.to_datetime(past_oil_cars_data['資料年月'], format='%Y%m')

history = {
	"data": [
		{
			"name": "燃油車數量",
			"data": []
		}
	]
}

latest_month = 10
for year in range(2012, 2022+1):
	past_oil_cars_data_single_year = past_oil_cars_data[past_oil_cars_data['資料年月'] == pandas.to_datetime(f'{year}{latest_month:02}', format='%Y%m')]
	# display(past_oil_cars_data_single_year)
	past_oil_cars_total_single_year = past_oil_cars_data_single_year['數量'].sum()
	history['data'][0]['data'].append( {'y': past_oil_cars_total_single_year, 'x': f'{year}-{latest_month}-01T00:00:00+08:00'} )

C:\Users\yucw\AppData\Local\Temp\ipykernel_32504\4272456501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_oil_cars_data['資料年月'] = pandas.to_datetime(past_oil_cars_data['資料年月'], format='%Y%m')


In [15]:
with open('cars_history.json', 'w', encoding='utf-8') as file:
	file.write( json.dumps(history, ensure_ascii=False, indent=4, default=int) )